In [14]:
import ray
import time
#ray.init()
@ray.remote
class Counter(object):
    def __init__(self):
        self.value = 0

    def increment(self):
        time.sleep(1)
        self.value += 1
        return self.value

    def get_counter(self):
        return self.value
    
@ray.remote
def f(counter):
    _ = [counter.increment.remote() for _ in range(4)]

        
        
counter = Counter.remote()
print("this is the actor identity \n",counter)

# Start some tasks that use the actor.
y = [f.remote(counter) for _ in range(3)]
print("\n This prints the ID of workers \n\n",y)


# Print the counter value.
#the program needs three seconds to compute before printing the result
time.sleep(3)
print(ray.get(counter.get_counter.remote()))

this is the actor identity 
 Actor(Counter,2762ccfcfbde749d0963c27201000000)

 This prints the ID of workers 

 [ObjectRef(7513710212de102affffffffffffffffffffffff0100000001000000), ObjectRef(eb7cccec83cc166cffffffffffffffffffffffff0100000001000000), ObjectRef(21711c35be2858f6ffffffffffffffffffffffff0100000001000000)]
12


In [15]:
import time
import dask

class Counter(object):
    def __init__(self):
        self.value = 0

    def increment(self):
        time.sleep(1)
        self.value += 1
        return self.value

    def get_counter(self):
        return self.value
    
    
    
def f(counter):
    _ = [counter.increment().result() for _ in range(4)]


from dask.distributed import Client

client = Client()
print("actor \n",client)

# Start some tasks that use the actor.
alist = [client.submit(Counter, actor=True).result() for _ in range(3)]
print("\n workers ",alist)

results = [f(a) for a in alist]
print("\n results", results)
print(alist)
print("\n this prints get_counter from increment \n")
print("\n This prints result from each worker \n",list(a.get_counter().result() for a in alist))
    
    

/Users/smanthony/opt/anaconda3/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 54019 instead
  http_address["port"], self.http_server.port


actor 
 <Client: 'tcp://127.0.0.1:54020' processes=4 threads=4, memory=8.00 GiB>

 workers  [<Actor: Counter, key=Counter-2e86e1a6-a097-44b7-93ed-86ff8076d750>, <Actor: Counter, key=Counter-43470d4e-2d4d-4972-bc00-4c655e1c502b>, <Actor: Counter, key=Counter-330fdc1b-057f-4c74-9a0a-00be75ceec30>]

 results [None, None, None]
[<Actor: Counter, key=Counter-2e86e1a6-a097-44b7-93ed-86ff8076d750>, <Actor: Counter, key=Counter-43470d4e-2d4d-4972-bc00-4c655e1c502b>, <Actor: Counter, key=Counter-330fdc1b-057f-4c74-9a0a-00be75ceec30>]

 this prints get_counter from increment 


 This prints result from each worker 
 [4, 4, 4]
